## Setup

In [1]:
from langchain_openai import AzureOpenAIEmbeddings 
from langchain.vectorstores.pgvector import PGVector 
from langchain_community.document_loaders import DirectoryLoader 
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.vectorstores import FAISS
from sqlalchemy import create_engine, text
from openai import AzureOpenAI 
from dotenv import load_dotenv
import os 

In [2]:
# Iniciando variáveis que serão usadas durante o processo de RAG e Chat
# Nome dos modelos no endpoint
embedding_model_name = 'embeddings-start'
openai_model_name = 'projeto-start'

# Importação das variáveis de ambiente que serão usadas em diversas funções (endpoint e chave)
load_dotenv()
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_API_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")

## RAG Local com FAISS

In [3]:
# Iniciando o client de embeddings
encoder = AzureOpenAIEmbeddings(
    azure_deployment=embedding_model_name,
    openai_api_version="2023-05-15",
)

In [4]:

# Lendo a base de conhecimento
loader = DirectoryLoader('../Documentação/', glob='**/**/*.txt')
docs = loader.load()

# Dividindo documento em chunks e gerando embeddings
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    separators = ["\n\n"]
)

split_doc = r_splitter.split_documents(docs)

vector = FAISS.from_documents(split_doc, encoder)

# Salvando base local
vector.save_local('faiss_index')

In [5]:

# Carregando FAISS db localmente
local_db = FAISS.load_local('faiss_index', encoder, allow_dangerous_deserialization= True)

## RAG com PGVector

In [6]:
# Inicializando chave de conexão com o banco
CONNECTION_STRING = os.getenv('CONNECTION_STRING')
COLLECTION_NAME = 'documentation'

# Iniciando o client de embeddings
encoder = AzureOpenAIEmbeddings(
    azure_deployment= embedding_model_name,
    openai_api_version="2023-05-15",
)


if COLLECTION_NAME != '':
        
    # Crie uma engine de conexão com o banco de dados
    engine = create_engine(CONNECTION_STRING)
            
    # Consultas SQL
    check_collection_query = text("""
        SELECT COUNT(*)
        FROM langchain_pg_collection
        WHERE name = :collection_name
        """)     
    

    # Consulta para excluir dados associados à coleção
    delete_collection_query = text("""
        DELETE FROM langchain_pg_collection
        WHERE name = :collection_name        
        """)  
    
    
    # Execute a consulta e verifique o resultado
    with engine.connect() as connection:
        
        transaction = connection.begin()
        
        try:
            result = connection.execute(check_collection_query, {'collection_name': COLLECTION_NAME})
            count = result.scalar()  # Obtém o valor único da consulta
            
            if count > 0:
                print(f"A coleção com o Nome '{COLLECTION_NAME}' existe. Sobrescrevendo....")
                
                connection.execute(delete_collection_query, {'collection_name': COLLECTION_NAME})
                
                transaction.commit() 
                
                #Carregar o conteúdo do arquivo de texto
                loader = TextLoader('../Documentação/documentation.txt', encoding='utf-8')
                documents = loader.load() 

                text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, separators=['\n\n'])

                texts = text_splitter.split_documents(documents) 
                
                # Gera e armazena embeddings no banco de dados usando a classe PGVector
                db = PGVector.from_documents( 

                embedding=encoder,
                documents=texts,
                collection_name=COLLECTION_NAME,
                connection_string=CONNECTION_STRING
                )           
            else:
                print(f"A coleção com o Nome '{COLLECTION_NAME}' não foi encontrada. Criando...")
                
                #Carregar o conteúdo do arquivo de texto
                loader = TextLoader('../Documentação/documentation.txt', encoding='utf-8')
                documents = loader.load() 

                text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, separators=['\n\n'])

                texts = text_splitter.split_documents(documents) 
                
                # Gera e armazena embeddings no banco de dados usando a classe PGVector
                db = PGVector.from_documents( 
                                
                embedding=encoder,
                documents=texts,
                collection_name=COLLECTION_NAME, 
                connection_string=CONNECTION_STRING
                )
                     
        except Exception as e:
            print(f"Erro ao executar a consulta: {e}")
            
    # Feche a conexão
    engine.dispose()

A coleção com o Nome 'documentation' não foi encontrada. Criando...


C:\Users\VITORFER\AppData\Roaming\Python\Python311\site-packages\langchain_community\vectorstores\pgvector.py:328: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata.Please note that filtering operators have been changed when using JSOB metadata to be prefixed with a $ sign to avoid name collisions with columns. If you're using an existing database, you will need to create adb migration for your metadata column to be JSONB and update your queries to use the new operators. 
  warn_deprecated(


# Prompt

In [22]:
# Criando prompt com as informações da base de conhecimento
pergunta = "Como me identifico?"

query_embedding = encoder.embed_query(pergunta)

retrieved_docs = db.similarity_search_by_vector(query_embedding, kwargs= 3)  #Para usar o FAISS ao inves do Postgres, basta mudar a variável db para local_db

# Ajuste para melhor leitura no prompt
contexto = "\n".join([doc.page_content for doc in retrieved_docs])

prompt = f"""
    Utilize o contexto a seguir para responder a pergunta em uma única sentença. Só responda caso a resposta seja encontrada no "contexto".
    pergunta : {pergunta}
    contexto : {contexto}
"""

# Chat

In [23]:
# Iniciando client de resposta da Azure OpenAI
client = AzureOpenAI(
    api_key = AZURE_OPENAI_API_KEY,  
    api_version = "2024-02-01",
    azure_endpoint = AZURE_OPENAI_API_ENDPOINT,
)


# Enviando a pergunta e recebendo a resposta
completion = client.chat.completions.create(
    model=openai_model_name,  # e.g. gpt-35-instant,
    temperature=0.8,    
    messages=[
        {
            "role": "system",
            "content": "Você é um assistente que responde de maneira educada somente perguntas sobre o funcionamento da API do Spotify. Caso a pergunta não tenha relação com a API do Spotify, peça desculpas, não responda e peça perguntas relacionadas à API do Spotify.",
        },
        {
            "role": "user",
            "content": prompt,
        },
    ],
)

print(completion.choices[0].message.content)

Sinto muito, mas não encontrei uma pergunta clara sobre o funcionamento da API do Spotify. Você tem alguma pergunta relacionada à API do Spotify?
